# Trabalho de Implementação - Heurísticas e Metaheurísticas
## Thiago Pádua de Carvalho - 2020007066

### Heurística Construtiva - Nearest Neighbour
A heurística construtiva escolhida foi a Nearest Neighbour. Sua estratégia é simples e consegue atingir uma boa aproximação do valor ótimo de maneira relativamente eficiente. A ideia é começar em um vértice arbitrário e, a partir dele, escolher o mais próximo que ainda não foi visitado. O algoritmo termina quando um ciclo hamiltoniano com nós correspondentes a todas as cidades é completo. A complexidade é O(n²) no pior caso, onde n corresponde ao número de cidades (pontos).

#### Implementação
Instâncias geométricas, onde as cidades correspondem a pontos no plano e a distância entre duas cidades depende de suas coordenadas apresentam a vantagem de possibilitar descartar rapidamente grandes grupos de arestas, aproveitando a geometria com estruturas de dados apropriadas.

##### **K-D Tree**
Em particular, para pontos no plano, é possível construir em tempo O(Nlog⁡N) uma estrutura de dados (ED) que permita responder a consultas de vizinhos mais próximos em tempo médio bem inferior a O(N) por consulta. A estrutura escolhida foi a k-d tree, com k = 2 dimensões.

Árvores desse tipo são similares a Binary Seach Trees. A diferença é que, em vez de comparar a chave de busca com a chave do nó atual, ela é comparada com a coordenada correspondente do nó atual. A cada nível da árvore, a dimensão da comparação é alternada. A árvore é construída recursivamente, de maneira que cada nó acrescentado divide o espaço em duas regiões menores, até que cada nó contenha um ponto. Desse modo, o número de operações necessárias para encontrar vizinhos mais próximos geralmente é em média O(log⁡N), quando a árvore á balanceada

![K-D Tree](KDtree.png)

Sendo assim, os passos da heurística são os seguintes:
```
1. Construir a k-d tree com as cidades
2. Inicializar o ciclo hamiltoniano com uma cidade arbitrária
3. Enquanto houver cidades não visitadas:
    4. Encontrar a cidade mais próxima da última cidade visitada
    5. Adicionar a cidade ao ciclo
    6. Remover a última cidade visitada da k-d tree
7. Adicionar a primeira cidade ao ciclo
```



### Análise dos Resultados

In [20]:
import os
import pandas as pd
import re
import subprocess
import time

In [21]:
class Report:
    def __init__(self):
        self.df = pd.DataFrame(columns=["Instance", "Heuristic", "Time (s)", "Result"])
        self.numeric_columns = ["Time (s)", "Result"]


    def generate_report(self, instance_path, heuristic="Nearest Neighbor"):
        start_time = time.time()
        result = subprocess.run(['./heuristic', instance_path], capture_output=True, text=True)
        end_time = time.time()
        elapsed_time = end_time - start_time

        instance_path = re.sub(r'\w+\/', "", instance_path)

        # Add new line to the dataframe
        self.df.loc[len(self.df)] = [instance_path, heuristic, elapsed_time, result.stdout.strip()]

        self.df[self.numeric_columns] = self.df[self.numeric_columns].apply(pd.to_numeric)

        return self.df


    def average_time(self):
        average_df = self.df.groupby('Instance', as_index=False)['Time (s)'].mean()
        average_df.rename(columns={"Time (s)": "Average Time (s)"}, inplace=True)
        return average_df


    def average_results(self):
        average_df = self.df.groupby('Instance', as_index=False)['Result'].mean()
        average_df.rename(columns={"Result": "Average Result"}, inplace=True)
        return average_df


    def reset_report(self):
        self.df = pd.DataFrame(columns=["Instance", "Heuristic", "Time (s)", "Result"])

In [22]:
report = Report()

In [27]:
directory = "instances/EUC_2D"
number_of_iterations = 100

for n in range(number_of_iterations):
    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        report.generate_report(filepath)

    NN_df = report.generate_report("instances/att48.tsp")

In [28]:
df_time = report.average_time()
df_result = report.average_results()

final_report = pd.merge(df_time, df_result, on="Instance")

display(final_report)
report.reset_report()

,Instance,Average Time (s),Average Result
0,att48.tsp,0.006985,46361.81
1,berlin52.tsp,0.006944,9969.57
2,kroA100.tsp,0.007194,36989.55
3,kroA150.tsp,0.007524,42017.56
4,kroA200.tsp,0.007929,46657.61
5,kroB100.tsp,0.007256,35705.27
6,kroB150.tsp,0.007619,40698.29
7,kroB200.tsp,0.007808,48792.03
8,kroC100.tsp,0.007232,33257.08
9,kroD100.tsp,0.007326,33641.34
